<a href="https://colab.research.google.com/github/MarcelodeFreitas/udemy_deep_learning_pytorch_python/blob/main/Tarefa_Iris_Tuning_dos_par%C3%A2metros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tarefa: Iris - Tuning dos parâmetros

## Etapa 1: Importação das bibliotecas

In [ ]:
!pip install skorch

In [ ]:
import pandas as pd
import torch.nn as nn
from skorch import NeuralNetClassifier
import torch
import torch.nn.functional as F
from sklearn.model_selection import GridSearchCV
torch.__version__

'1.4.0'

## Etapa 2: Base de dados

In [ ]:
torch.manual_seed(123)

In [ ]:
base = pd.read_csv('iris.csv')
previsores = base.iloc[:, 0:4].values
classe = base.iloc[:, 4].values

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
classe = labelencoder.fit_transform(classe)

In [ ]:
previsores = previsores.astype('float32')
classe = classe.astype('int64')

## Etapa 3: Construção do modelo

In [ ]:
class classificador_torch(nn.Module):
    def __init__(self, activation, neurons, initializer, dropout):
        super().__init__()
        self.dense0 = nn.Linear(4, neurons)
        initializer(self.dense0.weight)
        self.activation0 = activation
        self.dense1 = nn.Linear(neurons, neurons)
        initializer(self.dense1.weight)
        self.activation1 = activation
        self.dense2 = nn.Linear(neurons, 3)

        self.dropout = nn.Dropout(dropout)

    def forward(self, X):
        X = self.dense0(X)
        X = self.activation0(X)
        X = self.dropout(X)
        X = self.dense1(X)
        X = self.activation1(X)
        X = self.dropout(X)
        X = self.dense2(X)
        return X

In [ ]:
classificador_sklearn = NeuralNetClassifier(module = classificador_torch,
                                            criterion = torch.nn.CrossEntropyLoss,
                                            train_split = False)

## Etapa 4: Tuning

In [ ]:
params = {'batch_size': [10, 30],
          'max_epochs': [2000, 3000],
          'optimizer': [torch.optim.Adam, torch.optim.SGD],
          'module__activation': [F.relu, F.tanh, F.sigmoid],
          'module__neurons': [4, 8, 16],
          'module__initializer': [torch.nn.init.uniform_, torch.nn.init.normal_],
          'module__dropout': [0.2, 0.3]
          }

In [ ]:
grid_search = GridSearchCV(estimator = classificador_sklearn,
                           param_grid = params,
                           scoring = 'accuracy',
                           cv = 3)

In [ ]:
grid_search = grid_search.fit(previsores, classe)

Streaming output truncated to the last 5000 lines.
    250        0.2668  0.0154
    251        0.2439  0.0135
    252        0.2994  0.0142
    253        0.2761  0.0178
    254        0.2575  0.0142
    255        0.2709  0.0155
    256        0.1695  0.0158
    257        0.3624  0.0126
    258        0.2082  0.0158
    259        0.2201  0.0170
    260        0.2563  0.0124
    261        0.2715  0.0123
    262        0.2580  0.0133
    263        0.2013  0.0104
    264        0.2408  0.0110
    265        0.2429  0.0152
    266        0.2070  0.0131
    267        0.2393  0.0155
    268        0.2747  0.0129
    269        0.2227  0.0134
    270        0.1989  0.0156
    271        0.1960  0.0163
    272        0.2626  0.0159
    273        0.1710  0.0164
    274        0.1616  0.0162
    275        0.2577  0.0148
    276        0.1862  0.0161
    277        0.2348  0.0196
    278        0.2715  0.0161
    279        0.2163  0.0159
    280        0.2508  0.0157
    281        0.22

In [ ]:
melhores_parametros = grid_search.best_params_
melhor_precisao = grid_search.best_score_